# LangGraph + RAG (agent/pipeline) — komplett mal

Denne notebooken viser hvordan du kan modellere en **RAG-løsning** som en graf:

**Input → PII-maskering → Retrieval (MMR) → Stuffing → LLM → Output parsing → Policy-check → Svar**

Målet er å gjøre flyten:

- tydelig (for rapport/arkitektur)
- testbar
- sporbar (metadata, kilder, tracing)


## 1) Hvorfor LangGraph for RAG?

RAG alene kan bygges som en lineær chain. LangGraph blir nyttig når du vil ha:

- **routing** (skal vi gjøre retrieval eller direkte svar?)
- **retries** (hvis retrieval ikke finner noe)
- **guardrails** (PII/policy noder)
- **human-in-the-loop** (godkjenning før sending)
- **state** som holder: question, docs, sources, answer, flags


In [ ]:
# LangGraph + RAG (kode-mal)
# NB: Template. Krever pakker (langgraph/langchain/langchain-openai + vector store).

# from __future__ import annotations
# from typing import List, TypedDict, Literal, Optional
#
# from langgraph.graph import StateGraph, END
# from langchain_core.documents import Document
#
# # --- 1) State
# class RAGState(TypedDict, total=False):
#     question: str
#     masked_question: str
#     retrieved_docs: List[Document]
#     context: str
#     answer: str
#     sources: List[str]
#     policy_flags: List[str]
#
# # --- 2) Nodes
# def mask_pii(state: RAGState) -> RAGState:
#     # TODO: implementer ordentlig PII-maskering (regex/spacy)
#     q = state["question"]
#     masked = q.replace("@", "[at]")
#     return {"masked_question": masked}
#
# def retrieve(state: RAGState) -> RAGState:
#     # TODO: koble på din vectordb.as_retriever(search_type="mmr")
#     # docs = retriever.invoke(state["masked_question"])
#     docs: List[Document] = []
#     return {"retrieved_docs": docs}
#
# def stuff_context(state: RAGState) -> RAGState:
#     docs = state.get("retrieved_docs", [])
#     def title(d: Document) -> str:
#         return d.metadata.get("title") or d.metadata.get("source") or "unknown"
#
#     ctx_lines = []
#     sources = []
#     for d in docs:
#         sources.append(title(d))
#         ctx_lines.append(f"[{title(d)}] {d.page_content}")
#
#     return {
#         "context": "\n\n".join(ctx_lines),
#         "sources": sources,
#     }
#
# def generate(state: RAGState) -> RAGState:
#     # TODO: kall LLM (ChatOpenAI) med prompt som bruker {context}
#     # return {"answer": answer}
#     return {"answer": "Answer stub (would be LLM output)"}
#
# def policy_check(state: RAGState) -> RAGState:
#     # TODO: sjekk for sensitive ting i output, hallucination-policy, osv.
#     flags: List[str] = []
#     if "password" in state.get("answer", "").lower():
#         flags.append("contains_sensitive_keyword")
#     return {"policy_flags": flags}
#
# def route_after_policy(state: RAGState) -> Literal["ok", "needs_review"]:
#     if state.get("policy_flags"):
#         return "needs_review"
#     return "ok"
#
# def needs_review(state: RAGState) -> RAGState:
#     # human-in-the-loop i praksis
#     return {"answer": "[REVIEW REQUIRED] " + state.get("answer", "")}
#
# # --- 3) Build graph
# g = StateGraph(RAGState)
# g.add_node("mask_pii", mask_pii)
# g.add_node("retrieve", retrieve)
# g.add_node("stuff", stuff_context)
# g.add_node("generate", generate)
# g.add_node("policy", policy_check)
# g.add_node("review", needs_review)
#
# g.set_entry_point("mask_pii")
# g.add_edge("mask_pii", "retrieve")
# g.add_edge("retrieve", "stuff")
# g.add_edge("stuff", "generate")
# g.add_edge("generate", "policy")
#
# g.add_conditional_edges("policy", route_after_policy, {
#     "ok": END,
#     "needs_review": "review",
# })
# g.add_edge("review", END)
#
# app = g.compile()
#
# # --- 4) Run
# result = app.invoke({"question": "Hva er SLA for delayed shipping?"})
# result


## 2) Hvordan du gjør det “bachelor-sterkt”

- **Kilder/sporbarhet**: lagre `sources` fra metadata og legg dem i svaret
- **Kostkontroll**: top-k, chunk_size/overlap, maks output tokens
- **Policy**: PII-maskering før retrieval/LLM + policy-check etterpå
- **Observability**: logg timing + tokenbruk per node (LangSmith passer fint)

### Rapportsetning (paste-ready)

> “We modeled the RAG workflow as a graph to make control-flow, safety checks, and state transitions explicit. This improves debuggability, reproducibility, and supports production-oriented features such as human-in-the-loop and policy enforcement.”
